## This notebook calibrates the Cpx-Liq machine learning barometer used by Thermobar in the onnx framework
- We follow Petrelli, where a different random state is used for pressure and temperature
- Here, we use sklearn pipelines which we save to onnx

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import r2_score
import scipy.cluster.hierarchy as shc
from sklearn.cluster import AgglomerativeClustering
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingRegressor

%matplotlib inline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from scipy import stats
import warnings as w
import sys
sys.path.append("../../..")
import Thermobar as pt

In [5]:
out_train=pt.import_excel('GlobalDataset_Final_rev9_TrainValidation.xlsx',
                          sheet_name='Train_Thermobar_Format')
out_train_input=out_train['my_input']
Cpx_train=out_train['Cpxs']
Cpx_train_noID_noT=Cpx_train.drop(['Sample_ID_Cpx'], axis=1)
x_train=Cpx_train_noID_noT.values
y_train_T=out_train_input['T_K']
y_train_P=out_train_input['P_GPa']*10

## Saving onnx model

In [7]:
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Make pipeline
regr_T = ExtraTreesRegressor(n_estimators=450, criterion='mse', max_features=10, 
                             random_state=120) # random_state fixed for reproducibility

pipe = Pipeline([
    ('scalar', StandardScaler()),
    ('regressor', regr_T)
])

rf_model =pipe.fit(x_train, y_train_T)


initial_types=[('float_input', FloatTensorType([None, x_train.shape[1]]))]
model_onnx = convert_sklearn(rf_model, initial_types=initial_types)

with open("Petrelli2020_Cpx_only_Temp.onnx", "wb") as f:
    f.write(model_onnx.SerializeToString())
      

C:\Users\penny\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:396: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(


In [ ]:
    sess = rt.InferenceSession(str(Thermobar_dir/"Petrelli2020_Cpx_Liq_Temp.onnx"))
    #sess = rt.InferenceSession(Petrelli2020_Cpx_Liq_Temp.onnx)
    input_name = sess.get_inputs()[0].name
    label_name = sess.get_outputs()[0].name
    Pred_T_K = sess.run([label_name], {input_name: x_test.astype(np.float32)})[0]

    #df_stats, df_voting=get_voting_stats_ExtraTreesRegressor(x_test, ETR_Temp_P2020_Cpx_Liq)



    return Pred_T_K